In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

d:\App\anaconda\envs\toolkit\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#movies data
movies = tfds.load("movielens/100k-movies", split="train")
#ratings data
ratings = tfds.load("movielens/100k-ratings", split="train")

In [4]:
import pprint

for x in ratings.take(1).as_numpy_iterator():
  pprint.pprint(x)

tensorflow.python.data.ops.prefetch_op._PrefetchDataset

In [ ]:
def sort_ratings(ds):
    timestamps = []
    features_list = []

    for example in ds:
        timestamps.append(example['timestamp'])
        features_list.append(example)

    # convert list to tensor
    timestamps = tf.convert_to_tensor(timestamps)
    features_list = tf.stack(features_list)
    # sort index
    sorted_indices = tf.argsort(timestamps)
    # sort all data
    sorted_features = tf.gather(ds, sorted_indices)
    return tf.data.Dataset.from_tensor_slices(sorted_features)

In [5]:
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
})
movies = movies.map(lambda x: x["movie_title"])

In [ ]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [ ]:
movie_titles = movies.batch(2_000)
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

In [ ]:
unique_movie_titles = np.unique(list(movie_titles))
unique_user_ids = np.unique(list(user_ids))

unique_user_ids[:10]

In [ ]:
embedding_dimension = 32

user_model = tf.keras.Sequential([
    tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
    tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
])

In [ ]:
movie_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_movie_titles, mask_token=None),
  tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
])

In [ ]:
metrics = tfrs.metrics.FactorizedTopK(
  candidates=movies.batch(128).map(movie_model)
)

In [ ]:
task = tfrs.tasks.Retrieval(
  metrics=metrics
)

In [ ]:
from typing import Dict, Text

class MovielensModel(tfrs.Model):

  def __init__(self, user_model, movie_model):
    super().__init__()
    self.movie_model: tf.keras.Model = movie_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False):
    # pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # and pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    positive_movie_embeddings = self.movie_model(features["movie_title"])

    # computes the loss and the metrics.
    return self.task(user_embeddings, positive_movie_embeddings)

In [ ]:
model = MovielensModel(user_model, movie_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [ ]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [ ]:
model.fit(cached_train, epochs=5)

In [ ]:
model.evaluate(cached_test, return_dict=True)

In [ ]:
model.user_model.weights

In [ ]:
# create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
# recommends movies out of the entire movies dataset.
index.index_from_dataset(
  movies.batch(128).map(lambda title: (title, model.movie_model(title)))
)

# get recommendations.
_, titles = index(np.array(["21"]), k=3)
for x in titles[0]:
  print(x.numpy().decode('utf-8'))